In [40]:
import os
import csv
import json
import argparse
import requests
from calendar import monthrange
from datetime import datetime, timedelta

In [49]:
# Customise these values for automation. These can be taken as input args.

# start month to capture the metric. Inclusive. Format: '2020-01-01T00:00:00'
input_start_date = '2023-06-24T16:16:00'

# end month to capture the metric. Exclusive. Format: '2020-03-01T00:00:00'
input_end_date = '2024-05-25T16:17:00'

# grafana expression params
metricName = "request_duration_seconds"
saasEnv = "staging|prod"
apiRegex = ".*"
tenantName = "champagne"
statusCodeRegex = "4.*|5.*"

# grafana login session cookie. Obtained from browser logged-in session to 'https://thoughtspot.grafana.net/'
grafana_session = 'grafana_session=5ecd11e56bfb0d4ceae2e594450e80c7	'

# output directory where collected metric file per year will be saved.
output_dir = os.path.join(os.getcwd(), "data")

# grafana DS API query
# req and response format - https://grafana.com/docs/grafana/latest/developers/http_api/data_source/#query-a-data-source
url = "https://thoughtspot.grafana.net/api/ds/query"

# Metric data to be collected. DO NOT MODIFY UNLESS YOU KNOW WHAT YOU ARE DOING!
metric_keys = ('time', 'api', 'cluster', 'collection_level', 'error', 'method', 'org_id', 'rel_version', 'saas_env',
               'service_name', 'status', 'tenant_id', 'tenant_name', 'duration')

In [42]:
def parse_args():
    parser = argparse.ArgumentParser(description="Collect REST API usage metrics from Grafana")
    parser.add_argument('--input_start_date', type=str, default='2024-03-01T00:00:00',
                        help='start date for capturing metrics. Format: YYYY-MM-DDTHH:MM:SS')
    parser.add_argument('--input_end_date', type=str, default='2024-06-01T00:00:00',
                        help='end date for capturing metrics. Format: YYYY-MM-DDTHH:MM:SS')
    parser.add_argument('--metricName', type=str, default='request_duration_seconds',
                        help='prism: request_duration_seconds, tomcat: http_request_duration_us')
    parser.add_argument('--saasEnv', type=str, default='staging|prod',
                        help='prod|staging|dev')
    parser.add_argument('--apiRegex', type=str, default='.*',
                        help='regex to match api path')
    parser.add_argument('--tenantName', type=str, default='champagne',
                        help='tenant name')
    parser.add_argument('--statusCodeRegex', type=str, default='4.*|5.*',
                        help='api response status code regex to filter for: 2.*|4.*|5.*')
    parser.add_argument('--grafana_session', type=str, default='grafana_session=',
                        help='grafana_session=sessionId')
    return parser.parse_args()

In [43]:
def set_vars(args):
    global input_start_date, input_end_date, metricName, saasEnv, apiRegex, tenantName, statusCodeRegex, grafana_session
    input_start_date = args.input_start_date
    input_end_date = args.input_end_date
    metricName = args.metricName
    saasEnv = args.saasEnv
    apiRegex = args.apiRegex
    tenantName = args.tenantName
    statusCodeRegex = args.statusCodeRegex
    grafana_session = args.grafana_session

In [44]:
def makeQueryDict(queryRefId, queryExpr):
    query = {
        "refId": queryRefId,
        "datasourceId": 4,
        "expr": queryExpr,
        "interval": "600",
        "maxDataPoints": 1000,
        "instant": False,
        "range": True,
        "format": "table"
    }
    return query

In [45]:
# This method processes list of metric frames into CSV rows
def process_api_metric_frames(api_metric_frames_list, csv_writer):
    for api_metric_frame in api_metric_frames_list:
        timestamp_index = 0
        duration_value_index = 1
        api_metric_sample = dict.fromkeys(metric_keys, '')

        # print(json.dumps(api_metric_frame['schema']['fields'], indent=4))
        for field_index, api_metric_frame_field in enumerate(api_metric_frame['schema']['fields']):
            if "Time" == api_metric_frame_field['name']:
                timestamp_index = field_index

            if "labels" in api_metric_frame_field:
                frame_label_value_map = api_metric_frame_field['labels']
                # print(json.dumps(frame_label_value_map, indent=4))

                if "api" in frame_label_value_map:
                    api_metric_sample['api'] = frame_label_value_map['api']
                if "cluster" in frame_label_value_map:
                    api_metric_sample['cluster'] = frame_label_value_map['cluster']
                if "collection_level" in frame_label_value_map:
                    api_metric_sample['collection_level'] = frame_label_value_map['collection_level']
                if "error" in frame_label_value_map:
                    api_metric_sample['error'] = frame_label_value_map['error']
                if "method" in frame_label_value_map:
                    api_metric_sample['method'] = frame_label_value_map['method']
                if "org_id" in frame_label_value_map:
                    api_metric_sample['org_id'] = frame_label_value_map['org_id']
                if "rel_version" in frame_label_value_map:
                    api_metric_sample['rel_version'] = frame_label_value_map['rel_version']
                if "saas_env" in frame_label_value_map:
                    api_metric_sample['saas_env'] = frame_label_value_map['saas_env']
                if "service_name" in frame_label_value_map:
                    api_metric_sample['service_name'] = frame_label_value_map['service_name']
                if "status_code" in frame_label_value_map:
                    api_metric_sample['status'] = frame_label_value_map['status_code']
                if "status" in frame_label_value_map:
                    api_metric_sample['status'] = frame_label_value_map['status']
                if "tenant_id" in frame_label_value_map:
                    api_metric_sample['tenant_id'] = frame_label_value_map['tenant_id']
                if "tenant_name" in frame_label_value_map:
                    api_metric_sample['tenant_name'] = frame_label_value_map['tenant_name']

        if timestamp_index == 1:
            duration_value_index = 0

        #print(json.dumps(api_metric_frame['data']['values'], indent=4))
        #print("timestamp_index", timestamp_index)
        if len(api_metric_frame['data']['values']) > 0:
            for item_index, timestamp_value in enumerate(api_metric_frame['data']['values'][timestamp_index]):
                api_metric_sample['time'] = datetime.utcfromtimestamp(timestamp_value / 1000.0)
                api_metric_sample['duration'] = api_metric_frame['data']['values'][duration_value_index][item_index]

        # print(api_metric_sample)
        csv_writer.writerow(api_metric_sample)

In [46]:
# This method collect the api metric usage data from start_time to end_time
def collect_api_metric_usage(csv_writer, start_time, end_time, queryList):
    print(f'Collecting API metric data FROM "{start_time}" TO "{end_time}"')

    payload = json.dumps({
        "queries": queryList,
        "from": start_time,
        "to": end_time
    })
    headers = {
        'accept': 'application/json, text/plain, */*',
        'content-type': 'application/json',
        'Cookie': grafana_session
    }
    response = requests.request("POST", url, headers=headers, data=payload, allow_redirects=False)
    result = json.loads(response.text)

    if "results" not in result:
        print(json.dumps(result, indent=4))
        exit(1)

    for query in queryList:
        print(query["expr"])
        api_metric_frames_list = result['results'][query['refId']]['frames']
        process_api_metric_frames(api_metric_frames_list, csv_writer)

In [47]:
def start_rest_api_metric_collection(args):
    input_start_datetime = datetime.strptime(input_start_date, '%Y-%m-%dT%H:%M:%S')
    # start_datetime = input_start_datetime

    input_end_datetime = datetime.strptime(input_end_date, '%Y-%m-%dT%H:%M:%S')
    # end_datetime = start_datetime + timedelta(days=monthrange(start_datetime.year, start_datetime.month)[1]) - timedelta(milliseconds=1)

    start_time_str = input_start_datetime.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
    end_time_str = input_end_datetime.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"

    os.makedirs(output_dir, exist_ok=True)
    output_file = f'{output_dir}/{input_start_datetime.year}_{input_start_datetime.month}_to_{input_end_datetime.year}_{input_end_datetime.month}_rest_api_metric.csv'

    # file_exists = os.path.isfile(output_file)
    with open(output_file, mode='w') as csv_file:
        print("data save path: " + output_file)
        writer = csv.DictWriter(csv_file, fieldnames=metric_keys, delimiter=',', quotechar='"',
                                quoting=csv.QUOTE_MINIMAL)
        writer.writeheader()

        # Define your list of queries
        queryList = []
        queryList.append(
            makeQueryDict(
                "prism-public-rest-api-v2-query",
                "{}{{saas_env=~\"{}\", api=~\"{}\", tenant_name=~\".*{}.*\", quantile=\"0.95\", status_code=~\"{}\"}}".format(metricName, saasEnv, apiRegex, tenantName, statusCodeRegex)))
        collect_api_metric_usage(writer, start_time_str, end_time_str, queryList)

In [50]:
# Program execution entry point
if __name__ == '__main__':
    args = None
    args = parse_args()
    set_vars(args)
    start_rest_api_metric_collection(args)

data save path: /content/data/2023_6_to_2024_5_rest_api_metric.csv
request_duration_seconds{saas_env=~"staging|prod", api=~".*api/rest/2.0.*", tenant_name=~".*champagne.*", quantile="0.95", status_code=~"4.*|5.*"}
